In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

import numpy as np
import functools as ft

batch_size = 256
epochs = 200
latent_dim = 256
num_samples = 10000
# http://www.manythings.org/anki/fra-eng.zipを解凍
data_path = "fra-eng/fra.txt"

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

lines = open(data_path).read().split("\n")

"""
for line in lines[:min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
"""

def preprocess():
    global input_texts
    global target_texts
    global input_characters
    global target_characters

    corpus = lines[:min(num_samples, len(lines) - 1)]
    splitted = np.array(list(map(lambda x: x.split("\t"), corpus))).T.tolist()
    input_texts = splitted[0]
    target_texts = splitted[1]

    p_input_char_sets = map(lambda x: set(x), input_texts)
    p_target_char_sets = map(lambda x: set(x), target_texts)
    input_characters = ft.reduce(lambda a, b: a.union(b), p_input_char_sets)
    target_characters = ft.reduce(lambda a, b: a.union(b), p_target_char_sets)

    target_characters = target_characters.union({"\t", "\n"})
    target_texts = list(map(lambda x: "\t" + x + "\n", target_texts))

def decode_sequence(input_seq):
    state_value = encoder_model.predict(input_seq)
    target_seq = np.zeros([1, 1, num_decoder_tokens])
    target_seq[0, 0, target_token_index["\t"]] = 1.

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + state_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros([1, 1, num_decoder_tokens])
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

preprocess()
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = "float32")
decoder_input_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = "float32")
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = "float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

############################################################################

encoder_inputs = Input(shape = [None, num_encoder_tokens])
encoder = LSTM(latent_dim, return_state = True)
encoder_output, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape = [None, num_decoder_tokens])
decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation = "softmax")
decoder_outputs_logit = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs_logit)
model.compile(optimizer = "rmsprop",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
    batch_size = batch_size,
    epochs = epochs,
    validation_split = 0.2)

############################################################################

# backend.clear_session()
model.save("s2s.h5")

############################################################################

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape = [latent_dim,])
decoder_state_input_c = Input(shape = [latent_dim,])
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

############################################################################

encoder_model.save("encoder_model.h5")
decoder_model.save("decoder_model.h5")

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

for seq_index in range(100):
    input_seq = encoder_input_data[seq_index:seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)